## 最適化に入る前に


In [ ]:
# 普通のリスト
list1 = [1, 2, 3, 4, 5]

# 各要素を2乗したリストを作る➀
list2 = []
for i in list1:
    list2.append(i**2)

list2

In [ ]:
# 各要素を2乗したリストを作る➁
list3 = [i**2 for i in list1]
list3

In [ ]:
# 辞書
# キー：値という関係が保持される
price_dict = {
    'apple':100,
    'banana':200,
    'orange':300
}

# キーを指定すると値を参照できる
price_dict['banana']

In [ ]:
# もう一つ作っておく
# 数量を表す辞書
qty_dict = {
    'apple':2,
    'banana':4,
    'orange':3
}

qty_dict['banana']

In [ ]:
# キーだけ、値だけを取り出すことも可能
fruits = list(qty_dict.keys())
fruits

In [ ]:
# 最適化で頻出！積の和をとる
# それぞれの果物について価格×数量を計算し、その合計を計算している
sum( price_dict[fruit] * qty_dict[fruit] for fruit in fruits )

## データの読み込みと加工
今回は、架空のお菓子のカロリーと価格が載ったデータを扱います。  
数理最適化によって、予算内でなるべくたくさんのカロリーを摂取することを目指します！

In [ ]:
import pandas as pd
from pulp import *      # 毎回「pulp.~」と書くのが面倒なのでこうしています

In [ ]:
# データの読み込み
df = pd.read_csv('knapsack_data.csv')
df

In [ ]:
# カロリーが高いのは？
df.sort_values('kcal', ascending=False)

In [ ]:
# 最適化で記述しやすいように、辞書にしておきます
snack = df.set_index('name').to_dict(orient='index')
snack

In [ ]:
# お菓子の名前と属性を指定すれば値を参照できます
print(snack['いもりこ']['kcal'])
print(snack['いもりこ']['price'])

## 最適化モデルの作成
いよいよ最適化に入ります。データと予算を渡すと解いた結果を返してくれる関数にしましょう

In [ ]:
def optimize(snack, budget=500):

    # モデルのインスタンス化
    model = LpProblem('knapsack', sense=LpMaximize)

    # 商品のリスト
    items = list(snack.keys())

    # 決定変数
    # お菓子iを入れるか否か
    x = {}
    for i in items:
        x[i] = LpVariable(f'x({i})', cat='Binary')

    # 制約条件
    # 入れるお菓子の価格の合計が予算を超えない
    model += lpSum( snack[i]['price'] * x[i] for i in items ) <= budget

    # 目的関数
    # 入れるお菓子のカロリーの合計
    model += lpSum( snack[i]['kcal'] * x[i] for i in items )

    # 解く
    status = model.solve()

    return x, model.objective.value()

In [ ]:
# 関数を使ってみる
x, TotalCal = optimize(snack, 500)
TotalCal

## 結果の確認

In [ ]:
# どのお菓子を入れるのか、辞書にする
solution = {}
for i in snack.keys():
    solution[i] = x[i].value()

solution

In [ ]:
# 最初のデータフレームに追加する
df['buy'] = df['name'].map(solution)
df

In [ ]:
# コスパを出してみましょう
# 多分コスパ高いお菓子が入りやすくなっている　→　貪欲法の話
df['kcal_per_price'] = df.kcal / df.price
df.sort_values('kcal_per_price', ascending=False)